In [48]:
import numpy as np
import pandas as pd
import dask.dataframe as dd

from pandarallel import pandarallel
pandarallel.initialize()

from IPython.display import HTML

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [28]:
path_shared = '~/Downloads/nfl-big-data-bowl-2021/{}'

games_df = pd.read_csv(path_shared.format('games.csv'))
plays_df = pd.read_csv(path_shared.format('plays.csv'))
players_df = pd.read_csv(path_shared.format('players.csv'))
# coverage_df = pd.read_csv(path_shared.format('coverages_week1.csv'))
track_df = pd.read_csv(path_shared.format('week1_norm.csv', usecols=['nflId', 'displayName', 'position', 'team_pos', 'x', 'y', 'v_x', 'v_y', 'v_mag', 'v_theta', 'a_x', 'a_y', 'a_mag', 'a_theta']))

# plays_df = pd.merge(plays_df, coverage_df, left_on=['gameId', 'playId'], right_on=['gameId', 'playId'])

In [29]:

# plays_df[plays_df.coverage.str.contains('Zone')].sort_values('offensePlayResult', ascending=False).head()

In [30]:
params = lambda: None # create an empty object to add params
params.a_max = 7
params.s_max = 9
params.reax_t = params.s_max/params.a_max
params.avg_ball_speed = 20
params.tti_sigma = 0.45
params.cell_length = 1
vars(params)

{'a_max': 7,
 's_max': 9,
 'reax_t': 1.2857142857142858,
 'avg_ball_speed': 20,
 'tti_sigma': 0.45,
 'cell_length': 1}

In [31]:
# deep lockett vs chiefs
# game_id = 2018122314
# play_id = 4239

# deep dissly vs bears
# game_id = 2018091700
# play_id = 3936

# deep middle insane gronk 
game_id = 2018090905
play_id = 2062

# game_id, play_id = random.choice(plays)

play_df = track_df[(track_df.playId == play_id) & (track_df.gameId == game_id)].sort_values(by = 'frameId')
play_df.head()

,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,team,team_pos,teamAbbr,route,time,los,x,y,dis,o,s,s_dir,s_dir_rad,v_x,v_y,v_theta,v_mag,a_old,a_x,a_y,a_theta,a_mag
317421,2018090905,2062,1,None,496735.0,Kareem Jackson,25.0,CB,away,DEF,HOU,NaN,2018-09-09T18:28:11.700Z,40,54.51,31.08,0.08,260.44,0.83,108.40,1.89,0.79,-0.26,-0.32,0.83,0.38,0.0,0.0,0.0,0.0
317434,2018090905,2062,1,None,0.0,Football,NaN,NaN,football,FTBL,FTBL,NaN,2018-09-09T18:28:11.700Z,40,40.18,29.52,0.00,NaN,0.00,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
317433,2018090905,2062,1,None,2558094.0,Zach Cunningham,41.0,ILB,away,DEF,HOU,NaN,2018-09-09T18:28:11.700Z,40,44.36,32.36,0.05,263.88,0.49,262.25,4.58,-0.49,-0.07,0.14,0.49,0.72,0.0,0.0,0.0,0.0
317432,2018090905,2062,1,None,2552490.0,Benardrick McKinney,55.0,ILB,away,DEF,HOU,NaN,2018-09-09T18:28:11.700Z,40,41.88,23.53,0.10,220.61,0.95,298.75,5.21,-0.83,0.46,-0.50,0.95,0.03,0.0,0.0,0.0,0.0
317430,2018090905,2062,1,None,2552261.0,Kevin Johnson,30.0,CB,away,DEF,HOU,NaN,2018-09-09T18:28:11.700Z,40,48.40,17.56,0.05,135.60,0.50,27.09,0.47,0.23,0.45,1.10,0.50,0.46,0.0,0.0,0.0,0.0


In [32]:
plays_df.loc[(plays_df.gameId == game_id) & (plays_df.playId == play_id)]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
439,2018090905,2062,(1:05) (Shotgun) T.Brady pass deep middle to R...,2,1,10,NE,play_type_pass,NE,41,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,6.0,14.0,01:05:00,69.0,NaN,NaN,C,28,28,1.248695,False


In [33]:
plays_df.sort_values('epa', ascending=False).head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
17665,2018123004,2619,(8:51) (Shotgun) P.Mahomes pass deep middle to...,3,3,10,KC,play_type_pass,KC,11,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"2 DL, 3 LB, 6 DB",TRADITIONAL,3.0,21.0,08:51:00,99.0,TAU,KC 11,C,89,89,8.629316,False
1647,2018091607,329,(10:00) (Punt formation) K.Byard pass deep rig...,1,4,6,TEN,play_type_pass,TEN,34,NaN,"0 RB, 2 TE, 0 WR,1 P,3 LB,1 LS,4 DB",NaN,5.0,"0 DL, 3 LB, 4 DB, 3 RB,1 QB",UNKNOWN,0.0,0.0,10:00:00,44.0,NaN,NaN,C,66,66,8.397043,False
12752,2018112509,2475,(10:47) B.Roethlisberger pass deep middle to J...,3,1,10,PIT,play_type_pass,PIT,3,SINGLEBACK,"1 RB, 2 TE, 2 WR",8.0,6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,10.0,10.0,10:47:00,107.0,NaN,NaN,C,97,97,8.207265,False
12011,2018112201,2897,(2:27) (Shotgun) D.Prescott pass deep middle t...,3,2,9,DAL,play_type_pass,DAL,10,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"2 DL, 4 LB, 5 DB",TRADITIONAL,13.0,17.0,02:27:00,100.0,NaN,NaN,C,90,90,7.953610,False
10298,2018111104,617,(3:10) (Shotgun) B.Bortles pass deep right to ...,1,3,3,JAX,play_type_pass,JAX,20,EMPTY,"1 RB, 1 TE, 3 WR",5.0,3.0,"3 DL, 2 LB, 6 DB",TRADITIONAL,0.0,7.0,03:10:00,90.0,NaN,NaN,C,80,80,7.775187,False


In [34]:
games_df[games_df.gameId==game_id]

,gameId,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
5,2018090905,09/09/2018,13:00:00,NE,HOU,1


In [ ]:
%%time

def get_field_df(play_frame_group):
    frame_df = play_frame_group.loc[(play_df.nflId!=0)]
    # frame_df.set_index('nflId')
    ball_start = frame_df.loc[frame_df.position=='QB'][['x', 'y']].iloc[0].round().to_numpy()
    frame_df = frame_df.loc[frame_df.position!='QB']
    frame_df['v_x_r'] = frame_df.a_x*params.reax_t+frame_df.v_x
    frame_df['v_y_r'] = frame_df.a_y*params.reax_t+frame_df.v_y
    frame_df['v_r_mag'] = np.linalg.norm(np.array([frame_df.v_x_r, frame_df.v_y_r]), axis=0)
    frame_df['v_r_theta'] = np.arctan(frame_df.v_y_r/frame_df.v_x_r).fillna(0)

    frame_df['x_r'] = frame_df.x + frame_df.v_x*params.reax_t + 0.5*frame_df.a_x*params.reax_t**2
    frame_df['y_r'] = frame_df.y + frame_df.v_y*params.reax_t + 0.5*frame_df.a_y*params.reax_t**2

    x = np.linspace(0.5, 119.5, 120)
    y = np.linspace(-0.5, 53.5, 55)
    y[0] = -0.2
    xx, yy = np.meshgrid(x, y)
    field_locs = np.stack((xx, yy)).reshape(2, -1).T  # (F, 2)
    T = np.linspace(0.1, 4, 40) # (T,)

    player_teams = frame_df['team_pos'].to_numpy() # J,
    reaction_player_locs = frame_df[['x_r', 'y_r']].to_numpy() # (J, 2)
    reaction_player_vels = frame_df[['v_x_r', 'v_y_r']].to_numpy() #(J,2)

    # tofs = np.linalg.norm(field_locs-ball_start, axis=1)/params.avg_ball_speed # (F,)

    int_d_vec = field_locs[:, None, :] - reaction_player_locs #F, J, 2
    int_d_mag = np.linalg.norm(int_d_vec, axis=2) # F, J
    int_s0 = np.clip(np.sum(int_d_vec*reaction_player_vels, axis=2)/int_d_mag, -params.s_max, params.s_max) #F, J,  #taking norm between vectors int_d and player velocity

    t_lt_smax = (params.s_max-int_s0)/params.a_max  #F, J,
    d_lt_smax = t_lt_smax*((int_s0+params.s_max)/2) #F, J,
    d_at_smax = int_d_mag - d_lt_smax               #F, J,
    t_at_smax = d_at_smax/params.s_max              #F, J,
    t_tot = t_lt_smax+t_at_smax                     #F, J,

    # int_dT = tofs[:, None] - t_tot                  #F, J, T
    # int_dT = int_d_mag - T[:, None, None]           #T, F, J
    int_dT = np.add.outer(-t_tot, T)                 #F, J, T
    p_int = (1/(1. + np.exp( -np.pi/np.sqrt(3.0)/params.tti_sigma * int_dT) )) #F, J, T
    p_int = np.mean(p_int, axis=2) #F, J

    # F, T 
    # T, 3 (t, x, y )
    pass_trajs = np.tile(T, (len(field_locs), len(T), 1)) #(F, T, T)
    # todo NaN out (i, j, k) where j < k
    g = 10.72468
    reach_vecs = ball_start - field_locs  # (F, 2)
    dxy = np.linalg.norm(reach_vecs, axis=1) #F
    dxy_theta = np.arctan2(reach_vecs[:,1], reach_vecs[:,0])  #F
    # dxy_theta = np.arctan((ball_start[1]-field_locs[:, 1])/(ball_start[0]-field_locs[:, 0])) #F

    dx = -reach_vecs[:, 0] #F
    dy = -reach_vecs[:, 1] #F

    vxy = dxy[:, None]/T[None, :] #F, T
    vx = dx[:, None]/T[None, :]   #F, T
    vy = dy[:, None]/T[None, :]   #F, T
    vz_0 = (T*g)/2                #T

    #kinda slow af
    traj_locs_x = np.floor((ball_start[0]+vx[:, :, None]*pass_trajs))+0.5 # F, T, T
    traj_locs_y = np.floor((ball_start[0]+vy[:, :, None]*pass_trajs))+0.5 # F, T, T
    traj_locs_z = 2.0+vz_0[None, :, None]*pass_trajs-0.5*g*pass_trajs*pass_trajs #F, T, T
    # note that idx (i, j, k) into below arrays is invalid when j < k
    traj_locs_x_idx = np.clip((ball_start[0]+vx[:, :, None]*T), 0, len(x)-1).astype(int) # F, T, T
    traj_locs_y_idx = np.clip((ball_start[1]+vy[:, :, None]*T), 0, len(y)-1).astype(int) # F, T, T
    traj_locs_z = 2.0+vz_0[None, :, None]*pass_trajs-0.5*g*pass_trajs*pass_trajs #F, T, T
    path_idxs = np.ravel_multi_index(np.stack((traj_locs_y_idx, traj_locs_x_idx)).reshape(2, -1), xx.shape)  # (F*T*T,)
    probs = p_int[path_idxs] # F*T*T, J
    probs = probs.reshape((*traj_locs_x_idx.shape, len(reaction_player_locs)))  # F, T, T, J
    norm_factor = np.maximum(1., probs.sum(axis=-1))  #F, T, T
    probs_norm = (probs/norm_factor[..., None])  #F, T, T, J

    total_probs = np.sum(probs_norm, axis=-1)  # F, T, T
    compl_total_probs = 1-total_probs  # F, T, T
    remaining_compl_probs = np.cumprod(compl_total_probs, axis=-1)  # F, T, T
    off_probs = np.sum(probs_norm, axis=-1, where=(player_teams=='OFF'))
    def_probs = np.sum(probs_norm, axis=-1, where=(player_teams=='DEF'))
    both_probs = np.dstack((off_probs, def_probs))  # F, T, T, 2

    # maximum 0 because if it goes negative the pass has been caught by then and theres no residual probability

    shift_compl_cumsum = np.roll(remaining_compl_probs, 1, axis=-1)  # F, T, T
    shift_compl_cumsum[:, :, 0] = 1
    completion_prob_dt = shift_compl_cumsum * total_probs  # F, T, T
    completion_prob = np.cumsum(completion_prob_dt, axis=-1)  # F, T, T

    completion_prob_off_dt = shift_compl_cumsum * off_probs  # F, T, T
    completion_prob_def_dt = shift_compl_cumsum * def_probs  # F, T, T
    completion_prob_off = np.cumsum(completion_prob_off_dt, axis=-1)  # F, T, T
    completion_prob_def = np.cumsum(completion_prob_def_dt, axis=-1)  # F, T, T
    throw_p_int_off = np.einsum('ijj->ij', completion_prob_off)  # F, T
    throw_p_int_def = np.einsum('ijj->ij', completion_prob_def)  # F, T
    throw_p_int = np.einsum('ijj->ij', completion_prob)  # F, T
    field_p_int_off = throw_p_int_off.mean(axis=1)  # F,
    field_p_int_def = throw_p_int_def.mean(axis=1)  # F,
    field_p_int = throw_p_int.mean(axis=1)  # F,
    assert np.allclose(field_p_int, field_p_int_off + field_p_int_def)
    assert np.all(field_p_int <= 1) and np.all(field_p_int >= 0)
    assert np.all(field_p_int_off <= 1) and np.all(field_p_int_off >= 0)
    assert np.all(field_p_int_def <= 1) and np.all(field_p_int_def >= 0)
    field_p_no_int = 1-field_p_int

    #     #### Toy example
    #     total_probs = [0, 0, 0.1, 0.2, 0.8, 0.8]
    #     c_total_probs=[1, 1, 0.9, 0.8, 0.2, 0.2]
    #     0.1 + 0.9*0.2 + 0.72 * 0.8 + 0.144*0.8 = 0.9712
    #     cumsum =      [0, 0, 0.1, 0.3, 1.1, 1.9]
    #     compl =       [1, 1, 0.9, 0.7, 0, 0]
    #     adjust_compl_prob =        [0, 0, 0.1, 0.28, 0.84, 0.84]
    #     ####
    #     integ_stop = (cumsum >= 1).argmin(axis=-1)  # F, T

    field_df = pd.DataFrame({
        'ball_start_x': ball_start[0],
        'ball_start_y': ball_start[1], 
        'ball_end_x': field_locs[:,0],
        'ball_end_y': field_locs[:,1],
        'p_mass_1': (((field_p_int_off-field_p_int_def)+1.)/2.).round(3),
        'p_mass_2': field_p_no_int.round(3),
        # 'p_mass_players': p_int_norm,
    })
    return field_df
# play_df.loc[play_df.frameId==1]
# play_df.loc[play_df.frameId==1].apply(get_field_df)
# get_field_df(play_df.loc[play_df.frameId==1])
field_dfs = play_df.groupby(['gameId', 'playId', 'frameId']).parallel_apply(get_field_df)
field_dfs = field_dfs.reset_index(3, drop=True).reset_index()
field_dfs

In [ ]:
from visualize import AnimatePlay
animated_play = AnimatePlay(play_df, 20, field_dfs)
HTML(animated_play.ani.to_jshtml())

In [ ]:
field_dfs.loc[(field_dfs.frameId==46)&(field_dfs.ball_end_x>60)&(field_dfs.ball_end_x<70)&(field_dfs.ball_end_y>10)&(field_dfs.ball_end_y<20)].sort_values('p_mass_1', ascending=False).head()

In [ ]:
# USELESS JUNK BELOW

In [ ]:
    # # field_df = pd.DataFrame({'ball_start_x': ball_start[0], 'ball_start_y': ball_start[1], 'ball_end_x': ball_end[0], 'ball_end_y': ball_end[1]})
    # field_df['ball_tof'] = np.sqrt((field_df.ball_start_x-field_df.ball_end_x)**2+(field_df.ball_start_y-field_df.ball_end_y)**2)/params.avg_ball_speed

    # def getPitchControl(field_row):
    #     frame_df['int_d_x'] = field_row['ball_end_x']-frame_df.x_r
    #     frame_df['int_d_y'] = field_row['ball_end_y']-frame_df.y_r
    #     frame_df['int_d_mag'] = np.linalg.norm(np.array([frame_df.int_d_x, frame_df.int_d_y]), axis=0)
    #     frame_df['int_d_theta'] = np.arctan(frame_df.int_d_y/frame_df.int_d_x).fillna(0)

    #     if frame_df['int_d_mag'].min() > field_row.ball_tof*params.v_max*1.2:
    #         field_row['cp_off'], field_row['cp_def'], field_row['cp_dead'] = 0, 0, 1
    #         return field_row

    #     frame_df['int_v0'] = (frame_df['v_x_r']*frame_df['int_d_x'] + frame_df['v_y_r']*frame_df['int_d_y'])/frame_df['int_d_mag']

    #     frame_df['d_lt_vm'] = (params.v_max**2-frame_df.int_v0**2)/(2*params.a_max)
    #     frame_df['t_lt_vm'] = (params.v_max-frame_df.int_v0)/(params.a_max)
    #     frame_df['d_at_vm'] = frame_df.int_d_mag-frame_df.d_lt_vm
    #     frame_df['t_at_vm'] = frame_df.d_at_vm/params.v_max
    #     frame_df['t_tot'] = frame_df.t_lt_vm+frame_df.t_at_vm
    #     frame_df['int_dT'] = field_row['ball_tof'] - frame_df.t_tot
    #     frame_df['P_int_T_j'] = (1/(1. + np.exp( -np.pi/np.sqrt(3.0)/params.tti_sigma * (frame_df.int_dT) ) )).round(2)

    #     # topOff = frame_df.loc[frame_df.team_pos=="OFF"].sort_values('P_int_T_j', ascending=False)['P_int_T_j'].iloc[0]
    #     # topDef = frame_df.loc[frame_df.team_pos=="DEF"].sort_values('P_int_T_j', ascending=False)['P_int_T_j'].iloc[0]
    #     # total_p_int = 
    #     frame_df['norm_factor'] = np.minimum(1.0, 1.0/frame_df['P_int_T_j'].sum())
    #     frame_df['P_int_T_j_norm'] = frame_df.P_int_T_j*frame_df.norm_factor.round(2)
    #     field_row['cp_dead'] = 1.0-frame_df['P_int_T_j_norm'].sum()
    #     field_row['cp_off'] = frame_df.loc[frame_df.team_pos=='OFF']['P_int_T_j_norm'].sum()
    #     field_row['cp_def'] = frame_df.loc[frame_df.team_pos=='DEF']['P_int_T_j_norm'].sum()

    #     return field_row

    # # field_df.iloc[2278:2281].apply(getPitchControl, axis=1)
    # field_df = field_df.apply(getPitchControl, axis=1)
    # # frame_df[['displayName', 'teamAbbr', 't_tot', 'int_dT', 'P_int_T_j', 'norm_factor', 'P_int_T_j_norm', 'dead_ball', 'cp']].sort_values('P_int_T_j', ascending=False)
    # print(f'{frame_df.iloc[0].frameId}', end=' ')
    # return field_df

In [ ]:
# Select the game and play that you wish to see in week 1
import random

plays = list(track_df.groupby(['gameId', 'playId'], as_index=False).first()[['gameId', 'playId']].to_records(index=False))

game_id = 2018122314
play_id = 4239

# game_id, play_id = random.choice(plays)

play_df = track_df[(track_df.playId == play_id) & (track_df.gameId == game_id)].sort_values(by = 'time')
play_df.head()

In [ ]:
frame = 40
frame_df = play_df[(play_df.nflId!=0)&(play_df.frameId == 40)][['nflId', 'displayName', 'position', 'team_pos', 'x', 'y', 'v_x', 'v_y', 'v_mag', 'v_theta', 'a_x', 'a_y', 'a_mag', 'a_theta']]

In [ ]:
frame_df.head()

In [ ]:
a_max = 8
v_max = 9
reax_t = v_max/a_max

frame_df['v_x_r'] = frame_df.a_x*reax_t+frame_df.v_x
frame_df['v_y_r'] = frame_df.a_y*reax_t+frame_df.v_y
frame_df['v_r_mag'] = np.linalg.norm(np.array([frame_df.v_x_r, frame_df.v_y_r]), axis=0)
frame_df['v_r_theta'] = np.arctan(frame_df.v_y_r/frame_df.v_x_r).fillna(0)

frame_df['x_r'] = frame_df.x + frame_df.v_x*reax_t - 0.5*frame_df.a_x*reax_t**2
frame_df['y_r'] = frame_df.y + frame_df.v_y*reax_t - 0.5*frame_df.a_y*reax_t**2
frame_df

In [ ]:
frame_df['T_b'] = 2.7
frame_df['x_b'] = 75
frame_df['y_b'] = 20

frame_df['int_d_x'] = frame_df.x_b-frame_df.x_r
frame_df['int_d_y'] = frame_df.y_b-frame_df.y_r
frame_df['int_d_mag'] = np.linalg.norm(np.array([frame_df.int_d_x, frame_df.int_d_y]), axis=0)
frame_df['int_d_theta'] = np.arctan(frame_df.int_d_y/frame_df.int_d_x).fillna(0)

frame_df

In [ ]:
frame_df['int_v0'] = (frame_df['v_x_r']*frame_df['int_d_x'] + frame_df['v_y_r']*frame_df['int_d_y'])/frame_df['int_d_mag']

frame_df['d_lt_vm'] = (v_max**2-frame_df.int_v0**2)/(2*a_max)
frame_df['t_lt_vm'] = (v_max-frame_df.int_v0)/(a_max)
frame_df['d_at_vm'] = frame_df.int_d_mag-frame_df.d_lt_vm
frame_df['t_at_vm'] = frame_df.d_at_vm/v_max
frame_df['t_tot'] = frame_df.t_lt_vm+frame_df.t_at_vm
frame_df['int_dT'] = frame_df.T_b - frame_df.t_tot

frame_df.sort_values('t_tot', ascending=True)


In [ ]:
tti_sigma = 0.45
frame_df['P_int_T_j'] = (1/(1. + np.exp( -np.pi/np.sqrt(3.0)/tti_sigma * (frame_df.int_dT) ) )).round(2) 

frame_df[['displayName', 'team_pos', 'P_int_T_j']].sort_values('P_int_T_j', ascending=False)

In [ ]:
animated_play = AnimatePlay(play_df, 20)
HTML(animated_play.ani.to_jshtml())

In [ ]:


frame = 40
frame_df = play_df[(play_df.nflId!=0)&(play_df.frameId == 40)][['nflId', 'displayName', 'position', 'team_pos', 'x', 'y', 'v_x', 'v_y', 'v_mag', 'v_theta', 'a_x', 'a_y', 'a_mag', 'a_theta']]

frame_df = frame_df.copy()
frame_df = frame_df[['nflId', 'displayName', 'position', 'team_pos', 'x', 'y', 'v_x', 'v_y', 'v_mag', 'v_theta', 'a_x', 'a_y', 'a_mag', 'a_theta']]

start_loc = frame_df.loc[frame_df.position=='QB'][['x', 'y']].iloc[0]
frame_df = frame_df.loc[frame_df.position!='QB']

target_locs = np.array([np.array([x, y]) for y in range(52, -1, -1) for x in range(int(start_loc[0]), 120) ])
target_locs = target_locs + 0.5

start_locs = np.full_like(target_locs, start_loc)
tofs = np.linalg.norm(start_locs - target_locs, axis=1)/params.avg_ball_speed

frame_df['v_x_r'] = frame_df.a_x*reax_t+frame_df.v_x
frame_df['v_y_r'] = frame_df.a_y*reax_t+frame_df.v_y
frame_df['v_r_mag'] = np.linalg.norm(np.array([frame_df.v_x_r, frame_df.v_y_r]), axis=0)
frame_df['v_r_theta'] = np.arctan(frame_df.v_y_r/frame_df.v_x_r).fillna(0)

frame_df['x_r'] = frame_df.x + frame_df.v_x*reax_t - 0.5*frame_df.a_x*reax_t**2
frame_df['y_r'] = frame_df.y + frame_df.v_y*reax_t - 0.5*frame_df.a_y*reax_t**2

frame_df

In [ ]:
PCS = dict()

for i, (ball_x, ball_y) in enumerate(target_locs):
    # i, (ball_x, ball_y) = random.choice(list(enumerate(target_locs)))
    # print(i, ball_x, ball_y, tofs[i])

    df = frame_df.copy()
    df['int_d_x'] = ball_x-df.x_r
    df['int_d_y'] = ball_y-df.y_r
    df['int_d_mag'] = np.linalg.norm(np.array([df.int_d_x, df.int_d_y]), axis=0)
    df['int_d_theta'] = np.arctan(df.int_d_y/df.int_d_x).fillna(0)

    df['int_v0'] = (df['v_x_r']*df['int_d_x'] + df['v_y_r']*df['int_d_y'])/df['int_d_mag']

    df['d_lt_vm'] = (params.v_max**2-df.int_v0**2)/(2*params.a_max)
    df['t_lt_vm'] = (params.v_max-df.int_v0)/(params.a_max)
    df['d_at_vm'] = df.int_d_mag-df.d_lt_vm
    df['t_at_vm'] = df.d_at_vm/v_max
    df['t_tot'] = df.t_lt_vm+df.t_at_vm
    df['int_dT'] = tofs[i] - df.t_tot
    df['P_int_T_j'] = (1/(1. + np.exp( -np.pi/np.sqrt(3.0)/params.tti_sigma * (df.int_dT) ) )).round(2)

    topOff = df.loc[df.team_pos=="OFF"].sort_values('P_int_T_j', ascending=False)['P_int_T_j'].iloc[0]
    topDef = df.loc[df.team_pos=="DEF"].sort_values('P_int_T_j', ascending=False)['P_int_T_j'].iloc[0]

    # topOff-topDef
    PCS[(ball_x, ball_y)] = round(topOff-topDef, 2)

